In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-24"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11719,2024-10-24,Islândia Premier League,16:15,IR Reykjavik,Thorl,4.20,1.17,185.5,1.85,1.81,9.5,1.99,1.69,pOS9DGuR,0.238095,0.854701,0.540541,0.552486,0.092796,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,221.974,79.227895,0.356924,1.8,1.643168,0.912871,119.60,2.246,0.714619,0.318174,7.0,88,92,9.90,1.30,0.000,228.672,0.797964,0.015456,0.115289,0.00,0.000,inf,0.000000,0.0,0.000000,1.59,0.318,0.534591,0.487594,0.7,0.212406
11720,2024-10-24,Europa Eurocopa Feminina,15:30,San Giovanni F,Benfica F,1.02,10.60,129.5,1.83,1.83,-18.5,1.95,1.72,bXi2eLYH,0.980392,0.094340,0.546448,0.546448,0.074732,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,78,55,1.27,11.70,0.000,0.000,1.165935,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11721,2024-10-24,Irã Superliga,11:30,Mahgol Alborz,Kalleh,2.56,1.42,143.5,1.81,1.85,-1.5,1.83,1.83,MXLYUsY2,0.390625,0.704225,0.552486,0.540541,0.094850,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,159.706,34.930590,0.218718,1.2,1.643168,1.369306,150.45,1.988,0.773124,0.388895,-5.0,0,85,0.00,1.77,0.000,164.828,0.405076,0.015456,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,-3.64,-0.728,-0.576923,0.000000,0.0,0.000000
11722,2024-10-24,Irã Superliga,09:30,Palayesh Naft Abadan,Sanaye Hormozgan,1.72,1.95,151.5,1.83,1.83,-2.5,1.99,1.70,dWmtPipp,0.581395,0.512821,0.546448,0.546448,0.094216,212.098,66.149843,0.311883,1.2,1.643168,1.369306,291.84,2.658,1.157959,0.435651,-15.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,64,81,4.56,7.17,184.796,0.000,0.088629,0.000000,0.111144,2.25,0.450,1.600000,0.605206,0.6,-0.005206,0.00,0.000,inf,0.000000,0.0,0.000000
11723,2024-10-24,Hungria Nacional,14:00,Budapesti Honved Se,KTE-Duna,1.67,2.02,151.5,1.83,1.83,-3.5,2.03,1.66,48LoHwYJ,0.598802,0.495050,0.546448,0.546448,0.093852,279.360,159.539605,0.571090,0.6,1.341641,2.236068,528.48,3.668,2.177009,0.593514,-45.0,131.030,31.088813,0.237265,1.8,1.643168,0.912871,132.75,1.818,0.647356,0.356081,-18.0,72,75,7.34,1.77,234.196,196.086,0.134139,0.000000,0.141805,-1.87,-0.374,-1.791444,0.534055,0.6,0.065945,-0.02,-0.004,-255.000000,0.408696,0.3,-0.108696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11782,2024-10-24,China Cba,08:35,Shandong,Shenzhen,1.35,2.86,212.5,1.81,1.85,-7.5,1.96,1.72,fcCiljZR,0.740741,0.349650,0.552486,0.540541,0.090391,272.624,101.877294,0.373692,1.8,1.643168,0.912871,389.88,2.580,0.942815,0.365432,23.0,263.974,115.641688,0.438080,1.2,1.643168,1.369306,299.82,2.440,1.062285,0.435363,-14.0,108,114,3.61,2.63,210.684,343.392,0.507236,0.015456,0.092231,2.12,0.424,0.825472,0.578543,0.6,0.021457,-2.37,-0.474,-3.924051,0.435065,0.2,-0.235065
11783,2024-10-24,China Cba,08:35,Liaoning,Shanxi Zhongyu,1.14,4.61,213.5,1.80,1.86,-12.5,2.00,1.69,YsOrjC4F,0.877193,0.216920,0.555556,0.537634,0.094113,131.014,30.723292,0.234504,3.0,0.000000,0.000000,116.60,1.150,0.206277,0.

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11739,20:15,Uruguai Liga Uruguaia,Bigua,Malvin,1.71,Back Home
11755,11:00,Brasil Nbb,Botafogo,Pinheiros,1.92,Back Home
11766,15:30,Europa Euroliga,Milano,Anadolu Efes,1.80,Back Home
11768,21:05,Argentina Liga A,San Lorenzo,Gimnasia,1.54,Back Home
